# Uncollapsed Gibbs Sampling for Latent Dirichlet Allocation (LDA)

Below, we use text from posts on [beer.stackoverflow.com](https://archive.org/details/stackexchange) and an uncollapsed Gibbs Sampling algorithm for latent topic analysis.

We also use the perplexity measure to evaluate different values for K - the number of topics. As expected (and intuitively sensical), this measure suggests more topics fit the data better.

The results of LDA are presented as the most likely words for each topic. It is a matter of debate whether these topics are interpretable or useful, but hey, at least it's about beer!

In [1]:
from xml.etree import cElementTree as ET
import sys
from HTMLParser import HTMLParser
import time

# credit: http://stackoverflow.com/questions/753052/strip-html-from-strings-in-python
class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

posts = open('Posts.xml', 'r').read()

posts[1:100]

def remove_tags(text):
    return ''.join(ET.fromstring(text).itertext())

root = ET.fromstring(posts)
documents = []
t0 = time.time()
for child in root.findall('row')[0:1000]:
    text = None
    child_text = child.get('Body').encode('utf-8').strip()
    text = strip_tags(child_text)
    documents.append(text)
t1 = time.time()    

print 'Time to parse text: ' + str(t1 - t0)
print documents[0:2]
print 'Total number of documents: ' + str(len(documents))

Time to parse text: 0.113223075867
['I was offered a beer the other day that was reportedly made with citra hops. What are citra hops? Why should I care that my beer is made with them?', 'As far as we know, when did humans first brew beer, and where? Around when would you have been able to get your hands on something resembling a modern lager?']
Total number of documents: 1000


In [2]:
execfile('corpus.py')
execfile('document.py')

t0 = time.time()
corpus = Corpus(documents, '../Week1HW/stopwords.txt', 2)
t1 = time.time()

corpus.generate_document_term_matrix()
termlist = list(corpus.token_set)
print 'Sanity checks:'
print '\tTime to build corpus: ' + str(t1 - t0)
print '\tTerms in first document: ' + str(corpus.docs[0].tokens)
print '\tTotal words in corpus: ' + str(corpus.ntotal_tokens)
print '\tNumber docs in corpus: ' + str(corpus.N)
print '\tNumber of unique words in corpus: ' + str(len(corpus.token_set))

counting terms for doc: 0
counting terms for doc: 25
counting terms for doc: 50
counting terms for doc: 75
counting terms for doc: 100
counting terms for doc: 125
counting terms for doc: 150
counting terms for doc: 175
counting terms for doc: 200
counting terms for doc: 225
counting terms for doc: 250
counting terms for doc: 275
counting terms for doc: 300
counting terms for doc: 325
counting terms for doc: 350
counting terms for doc: 375
counting terms for doc: 400
counting terms for doc: 425
counting terms for doc: 450
counting terms for doc: 475
counting terms for doc: 500
counting terms for doc: 525
counting terms for doc: 550
counting terms for doc: 575
counting terms for doc: 600
counting terms for doc: 625
counting terms for doc: 650
counting terms for doc: 675
counting terms for doc: 700
counting terms for doc: 725
counting terms for doc: 750
counting terms for doc: 775
counting terms for doc: 800
counting terms for doc: 825
counting terms for doc: 850
counting terms for doc: 8

## Time metrics

Below are for K = 3

* For 1694 documents with 7942 unique tokens and 118617 words, the time to run the gibbs loop for 10 iterations was 7.4 minutes.
* For 200 documents 10 gibbs loop iterations took 13.88 seconds
* For 200 documents, 100 gibbs loop iterations took 143 seconds

So it makes sense that running gibbs for all documents will take 70 minutes with K = 3

#### Note: Computing time doesn't grow with K, only D.


## Algorithm

#### Initialization

    1. Randomly assign all words to a topic
    2. Initialize theta (D x K, document-specific topic probabilities): each theta_d is a k-dimensional sample from Dir(alpha + number of words in topic k)
    3. Initialize beta (K x V, topic-specific term probabilities): each beta_k is a V-dimensional sample from Dir(eta + number of times term v appeared in topic k)

#### Repeat

    1. for every word wi in N, it's current allocation is zi
        1.1. decrement -1 zi document-topic count and topic-term count
        1.2. Use theta and beta to calculate probability over topic for word (slide 14 lecture6)
        1.3. draw from multinomial with these probabilites - this is the new allocation zi for word wi
        1.4. increment +1 zi document-topic count and topic-term count
    2. update theta and beta

#### See `corpus.py` for this code: in `lda_gibbs()`

## Selection of K

To select the number of topics, one may use the perplexity measure. This measure is interpretable as the number of occurences of term v times it's total probability as estimated by theta and beta probability vectors, normalized by the total number of terms in the corpus.

In [7]:
def perplexity(corpus, probs, K):
    termlist = list(corpus.token_set)
    numerator_sum = 0
    for doci, doc in enumerate(corpus.docs):
        doc_prob_sum = 0
        for wordi, word in enumerate(doc.tokens):
            termidx = termlist.index(word)
            k_sum = 0
            # add to doc sum 1*prob of term for each k
            for k in range(K):
                k_sum += probs['theta'].item((doci, k))*probs['beta'].item((k, termidx))
            doc_prob_sum += np.log(k_sum)
        numerator_sum += doc_prob_sum
    return np.exp(-(numerator_sum/corpus.ntotal_tokens))

I realize the right thing to do here is parallelize, but it's not straight-forward to parrallelize gibbs sampling. 

In [16]:
# HERE BE THE CALL (commented because it takes over an hour to run for the whole set,
# so for the first 1000 documents took ~30 minutes)
# probs = corpus.lda_gibbs(K = 10, progress_interval = 25, iters = 100)

# Print the most likely words for every topic
beta = probs['beta']
K = beta.shape[0]
sorted_beta = np.zeros(beta.shape)

for k in range(0,K):
    print ''
    print 'Most likely terms for topic ' + str(k) + ':'
    sorted_beta[k,:] = np.argsort(beta[k,:])[::-1]
    for i in range(10):
        idx = int(sorted_beta[k,:][i])
        print termlist[idx] + ' with probability ' + str(beta.item((k, idx)))


Most likely terms for topic 0:
drink with probability 0.0523802103245
can with probability 0.036042161211
beer with probability 0.0359501071678
realli with probability 0.0255232138177
becaus with probability 0.0232588872648
ingredi with probability 0.0191920243353
probabl with probability 0.0190942741689
best with probability 0.0187138570491
someth with probability 0.0186840118498
want with probability 0.017960922743

Most likely terms for topic 1:
brew with probability 0.0391187770991
style with probability 0.0379814285584
thi with probability 0.0357220560253
lager with probability 0.0275471925447
ha with probability 0.0220679113848
ani with probability 0.021441369741
doe with probability 0.0212927481224
gener with probability 0.0209008570249
question with probability 0.0185651767617
process with probability 0.0179360316264

Most likely terms for topic 2:
beer with probability 0.054726525156
light with probability 0.0275541524642
find with probability 0.0257729981648
good with probab